In [1]:
import argparse
import io
import json
import os
from google.cloud import language_v1
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import numpy
import six
from datetime import datetime, timedelta
from kafka import KafkaConsumer

In [2]:
def analyze(txt):
    """Run a sentiment analysis request on text within a passed filename."""
    client = language_v1.LanguageServiceClient()
    content = txt
    document = language_v1.Document(
        content=content, type_=language_v1.Document.Type.PLAIN_TEXT
    )
    annotations = client.analyze_sentiment(request={"document": document})
    score = annotations.document_sentiment.score
    return score

In [3]:
scope = ["https://spreadsheets.google.com/feeds",'https://www.googleapis.com/auth/spreadsheets',"https://www.googleapis.com/auth/drive.file","https://www.googleapis.com/auth/drive"]
cerds = ServiceAccountCredentials.from_json_keyfile_name("youtubelive.json", scope)
client = gspread.authorize(cerds)
sheet = client.open("youtube_live").worksheet('nlp_comment') # change the cheet name here

def submit_result(video_id, username, comment, is_troll):
#     now = datetime.datetime.now()
#     dt = now.strftime("%m/%d/%Y, %H:%M:%S")
    dt = (datetime.now() - datetime(1899, 12, 30))/timedelta(days=1)
    row = [video_id, username, dt, comment, is_troll]
    sheet.append_row(row)

In [ ]:
consumer = KafkaConsumer(
 bootstrap_servers='ec2-65-1-100-39.ap-south-1.compute.amazonaws.com:9092',
value_deserializer = lambda v: json.loads(v.decode('utf-8'))
)
consumer.subscribe(topics='input_sleepy_youtube')
is_troll = 0
for message in consumer:
    try:
#   print ("%d:%d: v=%s" % (message.partition,
#                           message.offset,
#                           message.value))
        video_id = message.value.split(':::')[0]
        username = message.value.split(':::')[1]
        comment = message.value.split(':::')[2]
        sentiment_score = analyze(comment)
        if sentiment_score is not None:
            if sentiment_score <= -0.5 :
                is_troll = 1
                submit_result(video_id, username, comment, is_troll)
            else:
                is_troll = 0
            
            print(f'{video_id} {username} {comment} {sentiment_score}')
#             submit_result(video_id, username, comment, is_troll)
            
    except Exception as err:
        print(err)

luKStEOq_Hk Sumet Ketsri Negative comment เมื่อไหร่จะมา -0.5
luKStEOq_Hk Sumet Ketsri troll -0.6000000238418579
luKStEOq_Hk Sumet Ketsri spam -0.800000011920929
luKStEOq_Hk Sumet Ketsri มาละ 0.30000001192092896
luKStEOq_Hk Sumet Ketsri ปัญหาไม่มี -0.6000000238418579
luKStEOq_Hk Sumet Ketsri ไม่มีปัญหา 0.8999999761581421
luKStEOq_Hk Attasil Chitwirote OK 0.5
luKStEOq_Hk วัชรวิทย์ ลาพินี สวัสดีครับ 0.800000011920929
luKStEOq_Hk Attasil Chitwirote สวัสดีครับแม่ค้า 0.800000011920929
luKStEOq_Hk Attasil Chitwirote วันนี้มีของขายอะไรบ้างครับ 0.5
luKStEOq_Hk Sumet Ketsri ผมเป็น FC ครับ -0.5
luKStEOq_Hk Attasil Chitwirote I 0.10000000149011612
luKStEOq_Hk Attasil Chitwirote II 0.30000001192092896
luKStEOq_Hk Attasil Chitwirote III 0.30000001192092896
luKStEOq_Hk Gluai Namwaa  0.0
luKStEOq_Hk Attasil Chitwirote IV 0.30000001192092896
luKStEOq_Hk Attasil Chitwirote V 0.30000001192092896
luKStEOq_Hk Attasil Chitwirote VI 0.30000001192092896
luKStEOq_Hk Sumet Ketsri คุณทรงพลเป็น Admin ครับ แต่เกรี